In [ ]:
%pip install torch
%pip install transformers
%pip install pandas
%pip install ipywidgets

In [ ]:
import csv
import random
from typing import Iterator, Tuple
from tqdm import tqdm

def process_game(game: str) -> Iterator[Tuple[str, str, str, str]]:
    """
    Process a single game and yield (context, next_move, is_checkmate, outcome) tuples.
    """
    moves = game.split()
    outcome = moves[-1]
    moves = moves[:-1]  # Remove the outcome from the move list
    
    for i in range(len(moves)):
        context = " ".join(moves[:i])
        next_move = moves[i]
        is_checkmate = "1" if next_move.endswith("#") else "0"
        
        # For the last move, we know the outcome
        if i == len(moves) - 1:
            yield context, next_move, is_checkmate, outcome
        else:
            yield context, next_move, is_checkmate, ""

def preprocess_data(input_file: str, train_file: str, val_file: str, max_context_length: int = 50, val_split: float = 0.1):
    """
    Preprocess the input file and write (context, next_move, is_checkmate, outcome) tuples to train and val files.
    """
    # Open both output files
    with open(train_file, 'w', newline='') as train_outfile, open(val_file, 'w', newline='') as val_outfile:
        train_writer = csv.writer(train_outfile)
        val_writer = csv.writer(val_outfile)
        
        # Write headers
        headers = ['context', 'next_move', 'is_checkmate', 'outcome']
        train_writer.writerow(headers)
        val_writer.writerow(headers)
        
        # Count total lines for progress bar
        total_lines = sum(1 for _ in open(input_file, 'r'))
        
        # Process the input file line by line
        with open(input_file, 'r') as infile:
            for line in tqdm(infile, total=total_lines, desc="Processing games"):
                game = line.strip()
                for context, next_move, is_checkmate, outcome in process_game(game):
                    # Limit context to last `max_context_length` moves
                    context_moves = context.split()[-max_context_length:]
                    limited_context = " ".join(context_moves)
                    
                    # Decide whether to write to train or val file
                    if random.random() < val_split:
                        val_writer.writerow([limited_context, next_move, is_checkmate, outcome])
                    else:
                        train_writer.writerow([limited_context, next_move, is_checkmate, outcome])

In [ ]:
preprocess_data("out/grandmaster-truncated.txt", "out/training-data.csv", "out/validation-data.csv")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Config, GPT2Model

import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2Model

class ChessTransformer(nn.Module):
    def __init__(self, vocab_size, n_positions=50, n_embd=128, n_layer=2, n_head=2):
        super(ChessTransformer, self).__init__()
        
        self.config = GPT2Config(
            vocab_size=vocab_size,
            n_positions=n_positions,
            n_embd=n_embd,
            n_layer=n_layer,
            n_head=n_head
        )
        
        self.transformer = GPT2Model(self.config)
        self.move_head = nn.Linear(n_embd, vocab_size)
        self.checkmate_head = nn.Linear(n_embd, 1)
        self.outcome_head = nn.Linear(n_embd, 3)  # Win, Loss, Draw
        
    def forward(self, input_ids, attention_mask=None):
        outputs = self.transformer(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        
        # Use the second to last token's hidden state for prediction
        # This way, we're predicting based on the context, not just repeating the last move
        prediction_hidden_state = hidden_states[:, -2, :]
        
        # Predict next move
        move_logits = self.move_head(prediction_hidden_state)
        
        # Predict checkmate probability
        checkmate_logits = self.checkmate_head(prediction_hidden_state)
        
        # Predict game outcome
        outcome_logits = self.outcome_head(prediction_hidden_state)
        
        return move_logits, checkmate_logits, outcome_logits
        
class ChessTokenizer:
    def __init__(self):
        # Create a vocabulary of chess moves
        # This is a simplified version and should be expanded based on your data
        self.move_to_id = {"[PAD]": 0, "[UNK]": 1}
        self.id_to_move = {0: "[PAD]", 1: "[UNK]"}
        self.vocab_size = 2  # Start with PAD and UNK tokens

    def fit(self, moves):
        for move in moves:
            if move not in self.move_to_id:
                self.move_to_id[move] = self.vocab_size
                self.id_to_move[self.vocab_size] = move
                self.vocab_size += 1

    def encode(self, moves):
        return [self.move_to_id.get(move, self.move_to_id["[UNK]"]) for move in moves]

    def decode(self, ids):
        return [self.id_to_move.get(id, "[UNK]") for id in ids]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm.auto import tqdm

class ChessDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=50):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        context = row['context']
        if pd.isna(context) or context == '':
            context = []
        else:
            context = str(context).split()
        next_move = row['next_move']
        is_checkmate = float(row['is_checkmate'])
        outcome = row['outcome']

        # Tokenize input (context + next_move)
        input_ids = self.tokenizer.encode(context + [next_move])
        input_ids = input_ids[-self.max_length:]  # Keep only the last max_length tokens
        input_ids = [0] * (self.max_length - len(input_ids)) + input_ids  # Pad from the left

        # Create labels (next_move)
        labels = self.tokenizer.encode([next_move])[0]
        
        # Convert outcome to one-hot encoding
        outcome_label = torch.zeros(3, dtype=torch.float)
        if outcome == '1-0':
            outcome_label[0] = 1
        elif outcome == '0-1':
            outcome_label[1] = 1
        elif outcome == '1/2-1/2':
            outcome_label[2] = 1

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.long),
            'is_checkmate': torch.tensor(is_checkmate, dtype=torch.float),
            'outcome': outcome_label
        }

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import random

def fit_tokenizer(csv_file):
    data = pd.read_csv(csv_file)
    all_moves = []
    for _, row in data.iterrows():
        context = str(row['context']).split() if not pd.isna(row['context']) else []
        all_moves.extend(context)
        all_moves.append(row['next_move'])
    
    tokenizer = ChessTokenizer()
    tokenizer.fit(all_moves)
    return tokenizer

class ChessDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=50):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        context = str(row['context']).split() if not pd.isna(row['context']) else []
        next_move = row['next_move']
        is_checkmate = float(row['is_checkmate'])
        outcome = row['outcome']

        # Tokenize input (context + next_move)
        input_ids = self.tokenizer.encode(context + [next_move])
        input_ids = input_ids[-self.max_length:]  # Keep only the last max_length tokens
        input_ids = [0] * (self.max_length - len(input_ids)) + input_ids  # Pad from the left

        # Create labels (next_move)
        labels = self.tokenizer.encode([next_move])[0]
        
        # Convert outcome to one-hot encoding (as float)
        outcome_label = torch.zeros(3, dtype=torch.float)
        if outcome == '1-0':
            outcome_label[0] = 1.0
        elif outcome == '0-1':
            outcome_label[1] = 1.0
        elif outcome == '1/2-1/2':
            outcome_label[2] = 1.0

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.long),
            'is_checkmate': torch.tensor(is_checkmate, dtype=torch.float),
            'outcome': outcome_label
        }

def train_model(model, train_dataloader, val_dataloader, num_epochs, learning_rate, device):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    move_criterion = nn.CrossEntropyLoss()
    checkmate_criterion = nn.BCEWithLogitsLoss()
    outcome_criterion = nn.BCEWithLogitsLoss()

    total_steps = num_epochs * len(train_dataloader)
    progress_bar = tqdm(total=total_steps, desc="Training Progress")

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        
        for batch in train_dataloader:
            input_ids = batch['input_ids'].to(device)
            move_labels = batch['labels'].to(device)
            checkmate_labels = batch['is_checkmate'].to(device)
            outcome_labels = batch['outcome'].to(device)

            optimizer.zero_grad()

            move_logits, checkmate_logits, outcome_logits = model(input_ids)

            # Calculate losses
            move_loss = move_criterion(move_logits, move_labels)
            checkmate_loss = checkmate_criterion(checkmate_logits.squeeze(), checkmate_labels)
            outcome_loss = outcome_criterion(outcome_logits, outcome_labels)

            # Combine losses (you can adjust the weights)
            loss = move_loss + 0.1 * checkmate_loss + 0.1 * outcome_loss

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Update progress bar
            progress_bar.update(1)
            progress_bar.set_postfix({'epoch': epoch+1, 'loss': f'{loss.item():.4f}'})

        avg_loss = total_loss / len(train_dataloader)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_dataloader:
                input_ids = batch['input_ids'].to(device)
                move_labels = batch['labels'].to(device)
                checkmate_labels = batch['is_checkmate'].to(device)
                outcome_labels = batch['outcome'].to(device)

                move_logits, checkmate_logits, outcome_logits = model(input_ids)

                move_loss = move_criterion(move_logits, move_labels)
                checkmate_loss = checkmate_criterion(checkmate_logits.squeeze(), checkmate_labels)
                outcome_loss = outcome_criterion(outcome_logits, outcome_labels)

                loss = move_loss + 0.1 * checkmate_loss + 0.1 * outcome_loss
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_dataloader)
        
        # Print epoch results
        print(f"\nEpoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    progress_bar.close()
    return model


def calculate_random_baseline(dataloader, vocab_size, device):
    total_loss = 0
    move_criterion = nn.CrossEntropyLoss()
    checkmate_criterion = nn.BCEWithLogitsLoss()
    outcome_criterion = nn.BCEWithLogitsLoss()

    for batch in tqdm(dataloader, desc="Calculating random baseline"):
        batch_size = batch['labels'].size(0)
        
        # Generate random logits
        random_move_logits = torch.rand(batch_size, vocab_size).to(device)
        random_checkmate_logits = torch.rand(batch_size, 1).to(device)
        random_outcome_logits = torch.rand(batch_size, 3).to(device)

        # Calculate losses
        move_loss = move_criterion(random_move_logits, batch['labels'].to(device))
        checkmate_loss = checkmate_criterion(random_checkmate_logits.squeeze(), batch['is_checkmate'].to(device))
        outcome_loss = outcome_criterion(random_outcome_logits, batch['outcome'].to(device))

        loss = move_loss + 0.1 * checkmate_loss + 0.1 * outcome_loss
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

In [ ]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

print(f'Using device {get_device()}')

In [ ]:
# Initialize tokenizer and model
tokenizer = fit_tokenizer('out/training-data.csv')
model = ChessTransformer(vocab_size=tokenizer.vocab_size, n_positions=25, n_embd=128, n_layer=2, n_head=2)

# Load and prepare data
train_dataset = ChessDataset('out/training-data.csv', tokenizer, max_length=25)
val_dataset = ChessDataset('out/validation-data.csv', tokenizer, max_length=25)

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=128)

# Get the appropriate device
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Calculate random baseline loss
random_baseline_loss = calculate_random_baseline(train_dataloader, model.config.vocab_size, device)
print(f"Random Baseline Loss: {random_baseline_loss:.4f}")

# Train the model
trained_model = train_model(model, train_dataloader, val_dataloader, num_epochs=2, learning_rate=1e-3, device=device)

# Save the trained model
torch.save(trained_model.state_dict(), 'chess_transformer_model.pth')

In [ ]:
import torch
import torch.nn.functional as F
import random

def preprocess_input(move_sequence, tokenizer, max_length=25):
    """
    Preprocess the input move sequence for the model.
    """
    # Tokenize the input sequence
    input_ids = tokenizer.encode(move_sequence)
    
    # Truncate or pad the sequence to max_length
    if len(input_ids) > max_length:
        input_ids = input_ids[-max_length:]
    else:
        input_ids = [0] * (max_length - len(input_ids)) + input_ids
    
    return torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension

def predict_next_move(model, tokenizer, move_sequence, device, temperature=1.0, top_k=5):
    """
    Predict the next move, checkmate probability, and game outcome.
    """
    model.eval()  # Set the model to evaluation mode
    
    # Preprocess the input
    input_ids = preprocess_input(move_sequence, tokenizer).to(device)
    
    with torch.no_grad():
        move_logits, checkmate_logits, outcome_logits = model(input_ids)
    
    # Apply temperature to logits
    move_logits = move_logits / temperature
    
    # Get probabilities
    move_probs = F.softmax(move_logits, dim=-1)
    
    # Zero out the probability of the last move in the sequence
    last_move_id = tokenizer.encode([move_sequence[-1]])[0]
    move_probs[0, last_move_id] = 0
    
    # Normalize probabilities after zeroing out the last move
    move_probs = move_probs / move_probs.sum()
    
    # Get top-k moves
    top_k_probs, top_k_indices = torch.topk(move_probs, top_k)
    
    # Sample from top-k moves
    sampled_index = torch.multinomial(top_k_probs.squeeze(), 1).item()
    predicted_move_id = top_k_indices.squeeze()[sampled_index].item()
    predicted_move = tokenizer.decode([predicted_move_id])
    
    # Get the checkmate probability
    checkmate_prob = torch.sigmoid(checkmate_logits).item()
    
    # Get the game outcome probabilities
    outcome_probs = F.softmax(outcome_logits, dim=-1).squeeze()
    
    return predicted_move, checkmate_prob, outcome_probs, move_probs

def interpret_prediction(predicted_move, checkmate_prob, outcome_probs, move_probs, tokenizer):
    """
    Interpret the model's prediction in a human-readable format.
    """
    outcomes = ['Win', 'Loss', 'Draw']
    outcome_dict = {outcome: prob.item() for outcome, prob in zip(outcomes, outcome_probs)}
    most_likely_outcome = max(outcome_dict, key=outcome_dict.get)
    
    print(f"Predicted next move: {predicted_move}")
    print(f"Checkmate probability: {checkmate_prob:.2f}")
    print("Game outcome probabilities:")
    for outcome, prob in outcome_dict.items():
        print(f"  {outcome}: {prob:.2f}")
    print(f"Most likely outcome: {most_likely_outcome}")
    
    # Debugging information
    print("\nDebugging Information:")
    print(f"Vocabulary size: {len(tokenizer.move_to_id)}")
    print("Top 5 predicted moves:")
    top_moves = torch.topk(move_probs.squeeze(), 5)
    for i, (prob, idx) in enumerate(zip(top_moves.values, top_moves.indices)):
        move = tokenizer.decode([idx.item()])
        print(f"  {i+1}. {move} (probability: {prob.item():.4f})")

In [ ]:
# Load your trained model and tokenizer
# tokenizer = torch.load('chess_tokenizer.pth') # tokenizer is already in memory
model = ChessTransformer(vocab_size=len(tokenizer.move_to_id), n_positions=25, n_embd=128, n_layer=2, n_head=2)
model.load_state_dict(torch.load('chess_transformer_model.pth'))

device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
""

In [ ]:
# Example move sequence
move_sequence = ["e4", "e5", "Nf3", "Nc6", "Bb5"]

# Make multiple predictions
for _ in range(5):
    predicted_move, checkmate_prob, outcome_probs, move_probs = predict_next_move(model, tokenizer, move_sequence, device, temperature=0.8, top_k=5)
    print("\n--- New Prediction ---")
    interpret_prediction(predicted_move, checkmate_prob, outcome_probs, move_probs, tokenizer)